In [ ]:
class Experimentation(object):
    def __init__(self, configuration__, mode, variables, features_conf, regression_alg):
        self.configuration__ = configuration__
        self.variables = variables
        self.features_conf = features_conf
        self.regression_alg = regression_alg
        
        # declaring Bee
        self.geography = sb.Geography(configuration__, mode)
        self.sensors = sb.Sensors(configuration__, mode, path=configuration__['DATA_FOLDER'], delimit_geography=geography)
        self.features = sb.Features(configuration__, mode, Sensors=sensors, Geography=geography)
        
    def initiate(self, iterations, label):
        features = ['dow','day','week']
        
        self.scoring_dict = {}
        for var in variables:
            self.scoring_dict[var] = {}
            for fc in features_conf:
                self.scoring_dict[var][fc] = {}
                for ra in regression_alg:
                    
                    f = features.get_train_features(var)
                    if 'p' in fc:
                        features += var
                    if 'u' in fc:
                        features += configuration__['osm_line_objs']+configuration__['osm_point_objs']
                    if 's' in fc:
                        features += [x from x in configuration__['Sensors__variables'] if x!= var]
                    if 'd' in fc:
                        features += ['Index of Multiple Deprivation (IMD) Score', 'Income Score (rate)',
                                       'Employment Score (rate)', 'Education, Skills and Training Score',
                                       'Crime Score', 'Barriers to Housing and Services Score',
                                       'Living Environment Score',
                                       'Total population: mid 2012 (excluding prisoners)',
                                       'Population aged 16-59: mid 2012 (excluding prisoners)']
                    f = f['X'][features]
                    
                    if ra == 'rf':
                        regressor = RandomForestRegressor(n_estimators=200, max_depth=5, max_features=1)
                    elif ra == 'gb':
                        regressor = GradienBoostingRegressor(n_estimators=200, max_depth=5, max_features=1)
                    
                    r2,mse = [],[]
                    for i in range(1,iterations):
                        m = sb.Model(regressor).fit(f['X'], f['y'])
                        r2.append(m.r2)
                        mse.append(m.mse)
                    self.scoring_dict[var][fc][ra] = {'r2':np.mean(r2),'r2_std':np.std(r2),
                                                 'mse:'np.mean(mse),'mse:std':np.std(mse)}
                    print('[{} - {} - {}] r2: {} ~ std: {}'.format(var, fc, ra, np.mean(r2), np.std(r2)))
        return self
                        
                    
                
        

In [8]:
configuration__ = {
    'DATA_FOLDER':'/home/adelsondias/Repos/newcastle/air-quality/data_1week1/',
    'SHAPE_PATH':'/home/adelsondias/Repos/newcastle/air-quality/shape/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales.shp',
    'Sensors__frequency':'D',
    'Sensors__variables': ['NO2','Temperature','PM2.5'],
    'Sensors__threshold_callibration': {'Temperature':25, 'NO2':80, 'PM2.5':15},
    'Geography__filter_column':'msoa11nm',
    'Geography__filter_label':'Newcastle upon Tyne',
    'Geography__meshgrid':{'dimensions':[50,50], 'longitude_range':[-1.8, -1.51], 'latitude_range':[54.96, 55.05]},
    'osm_bbox': '(54.96,-1.8,55.05,-1.51)',
    'osm_line_objs': ['primary','trunk','motorway','residential'],
    'osm_point_objs': ['traffic_signals','crossing']
}